In [1]:
from torchtext.vocab import GloVe
embedding_glove = GloVe(name='6B', dim=50)

# Self-Attention

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
class SelfAttention(nn.Module):
    def __init__(self, num_heads, num_dim):
        super().__init__()

        self.num_heads = num_heads
        self.num_dim = num_dim

        self.toqueries = nn.Linear(self.num_dim, self.num_heads*self.num_dim, bias=False)
        self.tokeys = nn.Linear(self.num_dim, self.num_heads*self.num_dim, bias=False)
        self.tovalues = nn.Linear(self.num_dim, self.num_heads*self.num_dim, bias=False)
        self.unify_heads = nn.Linear(self.num_heads*self.num_dim, self.num_dim)

    def forward(self, x):
        num_batch, num_words, num_dim = x.shape
        num_heads = self.num_heads

        queries = self.toqueries(x).view(num_batch, num_words, num_heads, num_dim)
        keys = self.tokeys(x).view(num_batch, num_words, num_heads, num_dim)
        values = self.tovalues(x).view(num_batch, num_words, num_heads, num_dim)

        queries = queries.transpose(1, 2).contiguous().view(num_batch*num_heads, num_words, num_dim)
        keys = keys.transpose(1, 2).contiguous().view(num_batch*num_heads, num_words, num_dim)
        values = values.transpose(1, 2).contiguous().view(num_batch*num_heads, num_words, num_dim)

        queries = queries/(num_dim**(1/4))
        keys = keys/(num_dim**(1/4))

        raw_weights = torch.bmm(queries, keys.transpose(1, 2))
        weights = torch.softmax(raw_weights, dim=2)

        out = torch.bmm(weights, values).view(num_batch, num_heads, num_words, num_dim)
        out = out.transpose(1, 2).contiguous().view(num_batch, num_words, num_heads*num_dim)
        out = self.unify_heads(out)
        return out

# Transformer Block

In [3]:
class Transformer(nn.Module):
    def __init__(self, num_dim, num_heads):
        super().__init__()
        
        self.num_heads = num_heads
        self.num_dim = num_dim
        self.sa = SelfAttention(self.num_heads, self.num_dim)
        self.norm1 = nn.LayerNorm(self.num_dim)
        self.mlp = nn.Sequential(
                        nn.Linear(self.num_dim, 4*self.num_dim),
                        nn.ReLU(),
                        nn.Linear(4*self.num_dim, self.num_dim)
                        )
        self.norm2 = nn.LayerNorm(self.num_dim)

    def forward(self, x):
        out1 = self.sa(x)
        out2 = self.norm1(out1+x)
        out3 = self.mlp(out2)
        final = self.norm2(out3+out2)
        return final

# IMDB data pre-processing

In [107]:
!head 'aclImdb/train/pos/45_10.txt'

Home Room was a great movie if you've ever had drama in your life. It keeps you wanting to see more. Wondering what the secret Alicia is hiding. I think I watched that movie 6 times in a row and never lost interest. Plus I usually don't cry over movies but this one made me cry each time. I wish I could find more movies like that one. All in All I thought it was a great movie. The more you watch of it the more you become part of it. The very end is the part that really got me when she cried when getting her diploma, because it had her daughter's name on it. My heart felt as if it had shattered just then. And how her new friend came to comfort her when she hadn't gotten hers yet. I loved it so much.

In [108]:
from os import listdir
from os.path import isfile, join
trainposfiles = [join('aclImdb/train/pos/', f) for f in listdir('aclImdb/train/pos/') if isfile(join('aclImdb/train/pos/', f))]
trainnegfiles = [join('aclImdb/train/neg/', f) for f in listdir('aclImdb/train/neg/') if isfile(join('aclImdb/train/neg/', f))]
testposfiles = [join('aclImdb/test/pos/', f) for f in listdir('aclImdb/test/pos/') if isfile(join('aclImdb/test/pos/', f))]
testnegfiles = [join('aclImdb/test/neg/', f) for f in listdir('aclImdb/test/neg/') if isfile(join('aclImdb/test/neg/', f))]

In [109]:
train = []
test = []
for x in trainposfiles:
    f = open(x, "r")
    train.append({"review": f.read(), "sentiment": 1})
    f.close()
for x in trainnegfiles:
    f = open(x, "r")
    train.append({"review": f.read(), "sentiment": 0})
    f.close()
for x in testposfiles:
    f = open(x, "r")
    test.append({"review": f.read(), "sentiment": 1})
    f.close()
for x in testnegfiles:
    f = open(x, "r")
    test.append({"review": f.read(), "sentiment": 0})
    f.close()

In [45]:
import pandas as pd
df1 = pd.DataFrame(train)
df2 = pd.DataFrame(test)
df1.to_csv('train.csv', index=False)
df2.to_csv('test.csv', index=False)

In [32]:
import json
with open("train.json", "w") as trainfile:  
    json.dump(train, trainfile) 

with open("test.json", "w") as testfile:  
    json.dump(test, testfile) 

In [4]:
import spacy
spacy_en = spacy.load('en')
def tokenize(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [5]:
from torchtext.data import Field, TabularDataset, BucketIterator
review = Field(sequential=True, tokenize=tokenize, use_vocab=True, lower=True, batch_first=True, fix_length=1024)
sentiment = Field(sequential=False, use_vocab=False)

In [6]:
fields = {"review": ("review", review), "sentiment": ("sentiment", sentiment)}
train_data, test_data = TabularDataset.splits(
    path='/home/chirag_17bit012/Attention-Is-All-You-Get/data',
    format='csv',
    train='train.csv',
    test='test.csv',
    fields=[('review', review), ('sentiment', sentiment)],
    skip_header=True
)

In [7]:
review.build_vocab(train_data, vectors="glove.6B.50d")
batch_size = 8

In [8]:
train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data),
    batch_size=batch_size,
    sort=False
)

In [9]:
cnt = 0
for b in train_iterator:
    cnt+=1
    print(b.review.size())
    if cnt==10:
        break

torch.Size([8, 1024])
torch.Size([8, 1024])
torch.Size([8, 1024])
torch.Size([8, 1024])
torch.Size([8, 1024])
torch.Size([8, 1024])
torch.Size([8, 1024])
torch.Size([8, 1024])
torch.Size([8, 1024])
torch.Size([8, 1024])


In [10]:
vocab = review.vocab

In [11]:
len(vocab)

101513

In [12]:
embed = nn.Embedding(len(vocab), 50)
embed.weight.data.copy_(vocab.vectors)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.4180,  0.2497, -0.4124,  ..., -0.1841, -0.1151, -0.7858],
        ...,
        [-0.3564, -0.8063,  0.2048,  ..., -0.0914,  0.2320,  0.7523],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [13]:
embed(b.review).shape

torch.Size([8, 1024, 50])

In [14]:
import math

In [15]:
class Embedding(nn.Module):
    def __init__(self, embed, num_dim, device, max_len=10000):
        super().__init__()
        self.embed = embed
        self.device = device
        pe = torch.zeros(max_len, num_dim)
        for pos in range(max_len):
            for i in range(0, num_dim, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/num_dim)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/num_dim)))
                
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    def forward(self, x):
        num_words = x.shape[1]
        word_embedding = self.embed(x).to(self.device)
        postional_encoding = self.pe[ :, :num_words]
        postional_encoding.requires_grad = False
        postional_encoding = postional_encoding.to(self.device)
        return word_embedding + postional_encoding

# Classification Transformer
![image](http://peterbloem.nl/files/transformers/classifier.svg)

In [16]:
class ClassificationTransformer(nn.Module):
    def __init__(self, embed, device, num_blocks=1, num_dim=50, num_heads=8, num_classes=2):
        super().__init__()
        self.embbeding = Embedding(embed, num_dim, device)
        blocks = []
        for i in range(num_blocks):
            blocks.append(Transformer(num_dim, num_heads))
        self.transformer_blocks = nn.Sequential(*blocks)
        self.out = nn.Linear(num_dim, num_classes)
    def forward(self, x):
        embedded_input = self.embbeding(x)
        output = self.transformer_blocks(embedded_input)
        output = self.out(output.mean(dim=1))
        return F.log_softmax(output, dim=1)
    

In [18]:
import tqdm
import torch.optim as optim

num_heads = 8
num_classes = 2
num_blocks = 8

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embed = embed
model = ClassificationTransformer(embed, device, num_blocks=num_blocks, num_heads=num_heads, num_classes=num_classes).to(device)
opt = optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.NLLLoss()

epochs = 2

for epoch in range(1, epochs + 1):
    running_loss = 0.0
    running_corrects = 0
    model.train()
    #try:
    for train_obj in tqdm.tqdm(train_iterator):
        x = train_obj.review.to(device)
        y = train_obj.sentiment.to(device)
        opt.zero_grad()

        preds = model(x)
        loss = criterion(preds, y)
        loss.backward()
        opt.step()

        running_loss += loss.item() * batch_size
    epoch_loss = running_loss / len(train_iterator)

    val_loss = 0.0
    with torch.no_grad(): 
        for test_obj in test_iterator:
            inp = test_obj.review.to(device)
            tar = test_obj.sentiment.to(device)
            preds = model(inp)
            loss = criterion(preds, tar)
            val_loss += loss.item() * batch_size


    val_loss /= len(test_iterator)
    print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'.format(epoch, epoch_loss, val_loss))
    #except Exception as e:
    #    print(e)


print("Done")

100%|██████████| 3125/3125 [05:56<00:00,  8.76it/s]
Epoch: 2, Training Loss: 5.5603, Validation Loss: 5.5815
Done
